In [1]:
import numpy as np
import os
import h5py, pickle
from itertools import permutations
from sympy.combinatorics import Permutation

ens='cE211.044.112'

savepath=f'/p/project1/ngff/li47/code/projectData/05_moments/{ens}/data_merge/'
os.makedirs(savepath,exist_ok=True)

cfg2old=lambda cfg: cfg[1:]+'_r'+{'a':'0','b':'1','c':'2','d':'3'}[cfg[0]]
cfg2new=lambda cfg: {'0':'a','1':'b','2':'c','3':'d'}[cfg[-1]] + cfg[:4]
projs=['P0', 'Px', 'Py', 'Pz']
inserts=['tt', 'tx', 'ty', 'tz', 'xx', 'xy', 'xz', 'yy', 'yz', 'zz']
inds_trace=[i for i,ins in enumerate(inserts) if ins[0]==ins[1]]

xyzt2xyz0=lambda x: x if x!='t' else '0'
t=[insert for insert in inserts]; inserts_key=[f'=der:g{xyzt2xyz0(insert[1])}D{xyzt2xyz0(insert[0])}:sym=' for insert in t]

elements=[(sx,sy,sz,xyz) for sx in [1,-1] for sy in [1,-1] for sz in [1,-1] for xyz in permutations([0, 1, 2], 3)] # Permute first Flip next
def rotate_vec3(e,vec3): #xyzt=0123
    if vec3 in ['t']:
        return (1,vec3)
    sx,sy,sz,xyz=e; ivec3={'x':0,'y':1,'z':2}[vec3]; ivec3_new=xyz[ivec3]; vec3_new=['x','y','z'][ivec3_new]
    sign=[sx,sy,sz][ivec3_new]
    return (sign,vec3_new)
def rotate_proj(e,proj):
    if proj=='P0':
        return (1,proj)
    sx,sy,sz,xyz=e; det=sx*sy*sz*(1 if Permutation(xyz).is_even else -1)
    (sign,proj_new)=rotate_vec3(e,proj[-1])    
    return (sign*det,proj_new)
def rotate_insert(e,insert):
    s1,i1=rotate_vec3(e,insert[0]); s2,i2=rotate_vec3(e,insert[1])
    return (s1*s2,i1+i2 if i1+i2 in inserts else i2+i1)
def mom2name(mom):
    assert(np.all(mom==mom3pt2standard(mom)))
    return ','.join([str(ele) for ele in mom])


#!============== mom opearations ==============#
if True:
    def removeDuplicates_moms(moms):
        return [list(mom) for mom in {tuple(mom) for mom in moms}]
    
    def moms2dic(moms):
        dic={}
        for i,mom in enumerate(moms):
            dic[tuple(mom)]=i
        return dic
    def moms2list(moms):
        return [list(mom) for mom in moms]
    def mom2str(mom):
        return ','.join([str(ele) for ele in mom])
    def str2mom(momstr):
        return [int(ele) for ele in momstr.split(',')]
    
    def mom2msq(mom):
        assert(len(mom)==3)
        return mom[0]**2+mom[1]**2+mom[2]**2
    def getSortkey_mom(mom): # @mom2standard
        msq=mom2msq(mom)
        return (msq,-mom[2],-mom[1],-mom[0])
    def mom2standard(mom): # @getSortkey_mom
        return sorted([abs(mom[0]),abs(mom[1]),abs(mom[2])]) 
    
    elements_rot48=[(sx,sy,sz,xyz) for sx in [1,-1] for sy in [1,-1] for sz in [1,-1] for xyz in [(0,1,2),(0,2,1),(1,0,2),(1,2,0),(2,0,1),(2,1,0)]]
    def rotate_mom(e,mom):
        if len(mom)!=3:
            return rotate_mom(e,mom[:3])+rotate_mom(e,mom[3:])
        sx,sy,sz,xyz=e; ix,iy,iz=xyz; iix,iiy,iiz=tuple([ix,iy,iz].index(i) for i in range(3))
        return [sx*mom[iix],sy*mom[iiy],sz*mom[iiz]]
    def mom2moms(mom):
        return [list(mom) for mom in {tuple(rotate_mom(e,mom)) for e in elements_rot48}] 
    
    #==== for mom3pt=[p1,q] ====#
    def mom2n2qpp1(mom):
        p1x,p1y,p1z,qx,qy,qz=mom
        px,py,pz=p1x+qx,p1y+qy,p1z+qz
        
        q2=qx**2+qy**2+qz**2
        p2=px**2+py**2+pz**2
        p12=p1x**2+p1y**2+p1z**2
        return (q2,p2,p12)
    def mom2n2qpp1_sym(mom):
        (q2,p2,p12)=mom2n2qpp1(mom)
        return (q2,p2,p12) if p2>=p12 else (q2,p12,p2)
        
    def mom_exchangeSourceSink(mom):
        p1x,p1y,p1z,qx,qy,qz=mom
        px,py,pz=p1x+qx,p1y+qy,p1z+qz
        return [px,py,pz,-qx,-qy,-qz]
    
    def getSortkey_mom3pt_internal(mom):
        return (-mom[2+3],-mom[1+3],-mom[0+3],-mom[2],-mom[1],-mom[0])
    def mom3pt2standard_internal(mom):
        return min(mom2moms(mom),key=getSortkey_mom3pt_internal)
    def getSortkey_mom3pt(mom):
        p1x,p1y,p1z,qx,qy,qz=mom
        px,py,pz=p1x+qx,p1y+qy,p1z+qz
        
        q2=qx**2+qy**2+qz**2
        p2=px**2+py**2+pz**2
        p12=p1x**2+p1y**2+p1z**2
        
        if p2>p12:
            exchangeQ=0
            return (q2,-p2,p12,exchangeQ,getSortkey_mom3pt_internal(mom))
        momE=mom_exchangeSourceSink(mom)
        if p2<p12:
            exchangeQ=1
            return (q2,-p12,p2,exchangeQ,getSortkey_mom3pt_internal(momE))
        
        mom_std=mom3pt2standard_internal(mom)
        momE_std=mom3pt2standard_internal(momE)
        if getSortkey_mom3pt_internal(mom_std)<=getSortkey_mom3pt_internal(momE_std):
            exchangeQ=0
            return (q2,-p2,p12,exchangeQ,getSortkey_mom3pt_internal(mom))
        else:
            exchangeQ=1
            return (q2,-p12,p2,exchangeQ,getSortkey_mom3pt_internal(momE))
    def mom3pt2standard(mom):
        moms=mom2moms(mom)
        return min(moms,key=getSortkey_mom3pt)
    def mom3pt2standard_sym(mom):
        moms=mom2moms(mom)+mom2moms(mom_exchangeSourceSink(mom))
        return min(moms,key=getSortkey_mom3pt)

In [10]:
# check files complete
    
basepath='/p/scratch/jureap125/iona1/E112/nucl_threep/'
cfgs=[cfg2new(cfg) for cfg in os.listdir(basepath)]
cfgs.sort()

flag=True
for cfg in cfgs:
    cfg_old=cfg2old(cfg)
    path=f'{basepath}{cfg_old}/'
    files=os.listdir(path)
    
    tfs=list({int(file.split('_')[-2][2:]) for file in files if file.startswith('threep_')})
    tfs.sort()
    
    tf2Nsrc={}
    for tf in tfs:
        srcs=list({file.split('_')[1] for file in files if f'dt{tf}' in file})
        # print(tf,len(srcs))
        for src in srcs:
            fs=[file for file in files if f'dt{tf}' in file and src in file]
            assert(len(fs)==8)
        tf2Nsrc[tf]=len(srcs)
    
    if flag:
        tfs_global=tfs
        tf2Nsrc_global=tf2Nsrc
        flag=False
    assert(tfs==tfs_global)
    assert(tf2Nsrc==tf2Nsrc_global)

In [ ]:
flag_flip_q=False
flag_flip_Pxyz=False
flag_conjugate=True
def extractFile(f,mom):    
    p1=mom[:3]; q=mom[3:]
    assert(np.all(p1==[0,0,0]))
    msq='msq{:04}'.format(sum([x**2 for x in q]))
    moms=mom2moms(mom)
    
    t=list(f.keys()); t=[ele for ele in t if ele not in ['ffs']]; assert(len(t)==1); Nsrc_key=t[0]; Nsrc=int(Nsrc_key[4:])
    t=list(f[f'{Nsrc_key}/{msq}/{inserts_key[0]}'].keys()); assert(len(t)==1); proj_key=t[0]
    t=list(f[f'{Nsrc_key}/{msq}/{inserts_key[0]}/{proj_key}'].keys()); assert(len(t)==1); tf_key=t[0]
    t=list(f[f'{Nsrc_key}/{msq}/{inserts_key[0]}/{proj_key}/{tf_key}'].keys()); assert(len(t)==1); ud_key=t[0]
    moms_old=f[f'{Nsrc_key}/{msq}/{inserts_key[0]}/{proj_key}/{tf_key}/{ud_key}/mvec'][:]
    dic={}
    for i,m in enumerate(moms_old):
        if flag_flip_q:
            dic[tuple(-np.array(m))]=i
        else:
            dic[tuple(m)]=i
    inds=[dic[tuple(m[3:])] for m in moms]
    
    trajs=[traj.decode() for traj in f[Nsrc_key][msq][inserts_key[0]][proj_key][tf_key][ud_key]['trajs'][:]]
    if trajs[0][1]=='-':
        dic={}
        for i,tra in enumerate(trajs):
            cfg={'0':'a','1':'b','2':'c','3':'d'}[tra[0]]+tra[2:]
            dic[cfg]=i
    if trajs[0][-2]=='r':
        dic={}
        for i,tra in enumerate(trajs):
            cfg={'0':'a','1':'b','2':'c','3':'d'}[tra[-1]]+tra[:4]
            dic[cfg]=i
    inds_cfg=[dic[cfg] for cfg in cfgs]
    
    dat=np.array([f[f'{Nsrc_key}/{msq}/{insert_key}/{proj_key}/{tf_key}/{ud_key}/arr'][:] for insert_key in inserts_key])
    dat=dat[:,:,:,inds]
    dat=dat[:,inds_cfg]
    
    trace=np.sum(dat[inds_trace],axis=0)
    dat[inds_trace] -= trace[None,...]/4
    
    if flag_flip_Pxyz:
        if proj_key not in ['P0']:
            assert(proj_key in ['P4','P5','P6'])
            dat *= -1

    if flag_conjugate:
        dat=np.conj(dat)
    return dat

# verified agreeing with avgmore in post2avgmore
avgmoreQ=True
def convertData(dat,mom):
    dat=np.transpose(dat,[2,3,4,0,1])
    
    if avgmoreQ:
        assert(np.all(mom==mom3pt2standard(mom)))
        moms=mom2moms(mom)
        dic={}
        for i,m in enumerate(moms):
            dic[tuple(m)]=i

        e2ind_mom={}; e2inds_proj={}; e2signs_proj={}; e2inds_insert={}; e2signs_insert={}
        for e in elements:
            e2ind_mom[e]=dic[tuple(rotate_mom(e,mom))]
            
            sx,sy,sz,xyz=e; signs=[sx,sy,sz,1]
            ix,iy,iz=xyz; iix,iiy,iiz=tuple([ix,iy,iz].index(i) for i in range(3))
            xyzt=['x','y','z','t']
            xyzt2={'x':xyzt[ix],'y':xyzt[iy],'z':xyzt[iz],'t':'t'}
            det=sx*sy*sz*(1 if Permutation(xyz).is_even else -1)
            
            e2signs_proj[e]=np.array([1,sx*det,sy*det,sz*det])
            e2inds_proj[e]=[0,ix+1,iy+1,iz+1]
            
            e2signs_insert[e]=np.array([signs[xyzt.index(insert[0])]*signs[xyzt.index(insert[1])] for insert in inserts])
            e2inds_insert[e]=[xyzt2[insert[0]]+xyzt2[insert[1]] for insert in inserts]
            e2inds_insert[e]=[inserts.index(ele) if ele in inserts else inserts.index(ele[1]+ele[0]) for ele in e2inds_insert[e]]
            
        def get(e):
            t=dat[:,:,e2ind_mom[e]]
            
            t=t*e2signs_proj[e][None,None,:,None]
            t=t[:,:,e2inds_proj[e]]
            
            t=t*e2signs_insert[e][None,None,None,:]
            t=t[:,:,:,e2inds_insert[e]]
            
            return t
        dat=np.mean([get(e) for e in elements],axis=0)[:,:,None,:,:]
        
    return dat

In [ ]:
# def avgsrc():


def run(mom):
    path=f'{savepath}conn_{mom2name(mom)}.h5'
    
    
    for cfg in cfgs:
        cfg_old=cfg2old(cfg)
        path=f'{basepath}{cfg_old}/'
        files=os.listdir(path)
        print(files)
        break
    
run([0,0,0,0,0,0])

['threep_sx72sy54sz04st148_P4G5G1_P_dt23_up.h5', 'threep_sx44sy24sz23st145_P4_P_dt20_dn.h5', 'threep_sx38sy57sz61st189_P4_P_dt26_dn.h5', 'threep_sx71sy76sz60st104_P4G5G1_P_dt26_dn.h5', 'threep_sx38sy60sz100st005_P4G5G1_P_dt26_up.h5', 'twop_sx04sy89sz27st037.h5', 'threep_sx68sy12sz65st097_P4_P_dt23_dn.h5', 'twop_sx40sy11sz31st173.h5', 'threep_sx100sy37sz19st013_P4G5G1_P_dt26_dn.h5', 'threep_sx88sy69sz74st002_P4_P_dt23_up.h5', 'twop_sx43sy06sz96st030.h5', 'threep_sx94sy105sz90st212_P4_P_dt20_up.h5', 'threep_sx46sy67sz26st076_P4G5G2_P_dt26_up.h5', 'threep_sx15sy34sz87st205_P4G5G2_P_dt26_up.h5', 'threep_sx61sy24sz81st147_P4G5G1_P_dt23_dn.h5', 'twop_sx10sy101sz39st068.h5', 'threep_sx108sy19sz07st114_P4G5G1_P_dt26_dn.h5', 'threep_sx59sy23sz91st098_P4G5G1_P_dt26_dn.h5', 'threep_sx04sy89sz27st037_P4_P_dt20_up.h5', 'threep_sx07sy61sz104st022_P4G5G1_P_dt20_up.h5', 'threep_sx02sy28sz00st132_P4G5G3_P_dt20_dn.h5', 'twop_sx32sy62sz13st151.h5', 'threep_sx97sy81sz111st213_P4G5G3_P_dt14_up.h5', 'threep

In [5]:
max_mom2_pc=0
range_xyz=range(-int(np.sqrt(max_mom2_pc))-1,int(np.sqrt(max_mom2_pc))+2)
moms_pc=[[x,y,z] for x in range_xyz for y in range_xyz for z in range_xyz if x**2+y**2+z**2<=max_mom2_pc]

max_mom2_pf=0
range_xyz=range(-int(np.sqrt(max_mom2_pf))-1,int(np.sqrt(max_mom2_pf))+2)
moms_pf=[[x,y,z] for x in range_xyz for y in range_xyz for z in range_xyz if x**2+y**2+z**2<=max_mom2_pf]

moms=[pf+pc for pf in moms_pf for pc in moms_pc]
moms=[list(ele) for ele in set([tuple(mom3pt2standard(mom)) for mom in moms])]
moms=sorted(moms,key=getSortkey_mom3pt)

for imom,mom in enumerate(moms):
    print(f'{imom}/{len(moms)},{mom}',end='                 \r')
    run(mom)